In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pickle

In [7]:
## Load dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [8]:
#pre Process Data
## remove irrelevant features
data = data.drop(['RowNumber',"CustomerId",'Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [9]:
## encode categorical variable
le = LabelEncoder()
data['Gender'] = le.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [10]:
## one hot encoding 'Geography'
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
geo_enc = ohe.fit_transform(data[['Geography']])
geo_enc

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [11]:
geo_enc.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [12]:
ohe.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
geo_enc_df = pd.DataFrame(geo_enc.toarray(),columns=ohe.get_feature_names_out(['Geography']))

In [14]:
# combine ohe to original data

data = pd.concat([data.drop('Geography',axis=1),geo_enc_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [65]:
# save the encoders and scalers

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(le,file)
with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe,file)

In [66]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [67]:
## Dive dataset into  independent and dependent features
X = data.drop('Exited',axis=1)
Y = data['Exited']

## split the data in training and testing sets
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=43)

## scale down features
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)


In [68]:
X_train

array([[ 1.5178518 ,  0.90134069, -0.85641715, ...,  1.00125078,
        -0.57715782, -0.57850497],
       [ 1.0209782 ,  0.90134069, -0.28681727, ..., -0.99875078,
        -0.57715782,  1.72859362],
       [ 2.02507693, -1.1094584 , -0.09695065, ..., -0.99875078,
        -0.57715782,  1.72859362],
       ...,
       [ 0.48269847, -1.1094584 , -1.14121709, ..., -0.99875078,
         1.73262835, -0.57850497],
       [ 0.66902607,  0.90134069,  0.56758255, ..., -0.99875078,
        -0.57715782,  1.72859362],
       [-0.27296346,  0.90134069, -0.57161721, ..., -0.99875078,
         1.73262835, -0.57850497]])

In [69]:
X_test

array([[ 1.22800887, -1.1094584 ,  0.37771592, ..., -0.99875078,
         1.73262835, -0.57850497],
       [ 0.44129234,  0.90134069, -0.57161721, ...,  1.00125078,
        -0.57715782, -0.57850497],
       [ 0.34812854, -1.1094584 ,  3.22571532, ...,  1.00125078,
        -0.57715782, -0.57850497],
       ...,
       [-0.47999413,  0.90134069, -0.76148384, ...,  1.00125078,
        -0.57715782, -0.57850497],
       [ 0.3688316 , -1.1094584 ,  0.09291598, ...,  1.00125078,
        -0.57715782, -0.57850497],
       [-0.47999413,  0.90134069,  0.66251586, ..., -0.99875078,
        -0.57715782,  1.72859362]])

In [70]:
with open("scalar.pkl",'wb') as file:
    pickle.dump(scalar,file)

In [71]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [73]:
## Build Our ANN Model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid') ## op layer
])

/Users/sudeep/Documents/Gen Ai/ANN PRj/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [74]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [75]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.001)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [76]:
## Compile the model

model.compile(optimizer=opt,loss = loss,metrics=['accuracy'])


In [77]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d - %H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [78]:
## setup early stopping

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

##Training the model

history = model.fit(
    X_train,Y_train,validation_data = (X_test,Y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback]
)



Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 922us/step - accuracy: 0.7104 - loss: 0.5477 - val_accuracy: 0.8245 - val_loss: 0.4298
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step - accuracy: 0.8340 - loss: 0.4035 - val_accuracy: 0.8455 - val_loss: 0.3900
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 598us/step - accuracy: 0.8398 - loss: 0.3765 - val_accuracy: 0.8550 - val_loss: 0.3659
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 599us/step - accuracy: 0.8627 - loss: 0.3412 - val_accuracy: 0.8605 - val_loss: 0.3596
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 594us/step - accuracy: 0.8604 - loss: 0.3416 - val_accuracy: 0.8600 - val_loss: 0.3540
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 590us/step - accuracy: 0.8608 - loss: 0.3407 - val_accuracy: 0.8605 - val_loss: 0.3538
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 600us/step - accuracy: 0.8599 - loss: 0.3360 - val_accuracy: 0.8585 - val_loss: 0.3519
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 596us/step - accuracy: 0.8607 - loss: 0

In [79]:
model.save('model.h5')

In [80]:

%reload_ext tensorboard

In [81]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 19148), started 0:27:59 ago. (Use '!kill 19148' to kill it.)